In [ ]:
path = []


In [ ]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [3]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

NameError: name 'path' is not defined

In [3]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

NameError: name 'pnts' is not defined

In [4]:
import math
print(slp)

[]


In [6]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost
minibatch = []

In [5]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import time
import threading

In [10]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.8
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
        self.load_bst()
    
    def xgbmodel_bld(self,data,label):
        param = {'base_score' : 1, 'max_depth': 7, 'eta': 0.1, #'updater':'refresh',
            #'process_type': 'update',
            'refresh_leaf': True,
            'reshape': True,
            'reg_alpha': 3, 
            'silent': 1, 
            'objective': 'multi:softprob',     
            'num_class': 6}  
        param['nthread'] = 4
        param['eval_metric'] = 'merror'
        num_round = 500  
        dtrain = xgb.DMatrix(data, label=label)
        evallist = [(dtrain, 'train'), (dtrain, 'eval')]
        self.bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3, xgb_model = self.name)
        self.save_bst()

        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
#         print('act_values',act_values)
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
#         print(len(minibatch))
        bst_target = xgb.Booster({'nthread':4})
        bst_target.load_model(self.name)
        st = []
        tg = []
        for state, action, reward, next_state, done in minibatch:

            target = bst_target.predict(xgb.DMatrix(state))##target is Q matrix
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = (1-self.gamma)*reward + self.gamma * np.amax(t)
            st.append(state)
            tg.append(np.argmax(target[0]))
        self.memory.clear()
        st = np.array(st)
        st = np.reshape(st, [-1, 15])
        tg = np.array(tg)
        tg = np.reshape(tg, [-1,1])
        print(st.shape, tg.shape)
        self.xgbmodel_bld(st, tg)

        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = (np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1])))/175
#         print('reward',rew)
#         print('state',state)
        done = False
        if rew > 0.74:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.random.randint(175, size=15)/100
        return stt
    
def steer(GameOn):
    
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
#     agent.load_bst()
    count = 0
    bst_target = xgb.Booster({'nthread':4})
    st = []
    tg = []
    bst_target.load_model(agent.name)
    
    while GameOn:
        start = time.time()
        count += 1
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
#         reward = reward 
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
#         target = bst_target.predict(xgb.DMatrix(state))
#         if done:
#             target[0][action] = reward
#         else:
#             t = agent.bst.predict(xgb.DMatrix(next_state))[0]
#             target[0][action] = (1-agent.gamma)*reward + agent.gamma * np.amax(t)
#         st.append(state)
#         tg.append(np.argmax(target[0]))
        
#         if count % 250 == 0:
#             count = 0
#             st = np.array(st)
#             st = np.reshape(st, [-1, 15])
#             tg = np.array(tg)
#             tg = np.reshape(tg, [-1,1])
#             agent.xgbmodel_bld(st, tg)
#             if agent.epsilon > agent.epsilon_min:
#                 agent.epsilon *= agent.epsilon_decay
#             st = []
#             tg = []
#             bst_target = None
#             bst_target = xgb.Booster({'nthread':4})
#             bst_target.load_model(agent.name)
#             print("success")
        state = next_state
        if done and len(agent.memory) >= 50 or len(agent.memory) >= 500 :
            print("replaying at {} xgboost".format(len(agent.memory)))
            t1 = threading.Thread(target=agent.replay, args=())
            t1.start()
#         time.sleep(0.03)
        print("FPS: ", 1.0 / (time.time() - start))
        

In [ ]:
steer(True)

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:118: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  2432.890951276102
FPS:  9157.868995633187
FPS:  10205.119221411192
FPS:  10305.415233415233
FPS:  8943.078891257996
FPS:  9822.72599531616
FPS:  9915.612293144208
FPS:  9776.932400932401
FPS:  10837.994832041344
FPS:  12633.44578313253
FPS:  13148.288401253918
FPS:  10894.296103896104
FPS:  7294.441739130435
FPS:  11096.042328042327
FPS:  12671.61329305136
FPS:  15477.136531365313
FPS:  16912.516129032258
FPS:  17260.510288065845
FPS:  17924.37606837607
FPS:  9532.50909090909
FPS:  10230.009756097561
FPS:  12483.047619047618
FPS:  11366.677506775068
FPS:  15141.891696750903
FPS:  10205.119221411192
FPS:  12671.61329305136
FPS:  5849.796373779637
FPS:  9915.612293144208
FPS:  12052.597701149425
FPS:  13357.656050955415
FPS:  9532.50909090909
FPS:  14122.23569023569
FPS:  12052.597701149425
FPS:  8774.694560669455
FPS:  9619.963302752294
FPS:  9446.63063063063
FPS:  9532.50909090909
FPS:  11781.752808988764
FPS:  16070.130268199233
FPS:  19239.92660550459
FPS:  19152.07305936073
FP

FPS:  26.094690606841116
replaying at 506 xgboost
FPS:  72.45800366237087
replaying at 507 xgboost
FPS:  71.51169610584465
replaying at 508 xgboost
FPS:  158.3234183904575
replaying at 509 xgboost
FPS:  33.771107425240345
replaying at 510 xgboost
FPS:  228.88425648021828
replaying at 511 xgboost
FPS:  29.62539377586913
replaying at 512 xgboost
FPS:  1548.2849760059062
replaying at 513 xgboost
FPS:  84.26865971510658
replaying at 514 xgboost
FPS:  85.7783504100456
replaying at 515 xgboost
FPS:  127.91021926748193
replaying at 516 xgboost
FPS:  58.85833766015072
replaying at 517 xgboost
FPS:  129.23444769681097
replaying at 518 xgboost
FPS:  1039.4805452292442
replaying at 519 xgboost
FPS:  329.84460522176784
replaying at 520 xgboost
FPS:  397.9415559772296
replaying at 521 xgboost
FPS:  36.58418812364803
replaying at 522 xgboost
FPS:  900.0652360515021
replaying at 523 xgboost
FPS:  84.82423605072097
replaying at 524 xgboost
FPS:  85.16526223882718
replaying at 525 xgboost
FPS:  134.774

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-29:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-85:
T

1392.531208499336
replaying at 596 xgboost
FPS:  1.9377480047585498
replaying at 597 xgboost
FPS:  85.8362802881467
replaying at 598 xgboost
FPS:  28.27169597659699
replaying at 599 xgboost
FPS:  8.236884585927221
replaying at 600 xgboost
FPS:  524.288
replaying at 601 xgboost
FPS:  1121.4716577540107
replaying at 602 xgboost
FPS:  46.40127444906629
replaying at 603 xgboost
FPS:  920.0052643123493
replaying at 604 xgboost
FPS:  268.1780051150895
replaying at 605 xgboost
FPS:  1691.2516129032258
replaying at 606 xgboost
FPS:  327.11776633910466
replaying at 607 xgboost
FPS:  35.12964529502911
replaying at 608 xgboost
FPS:  3.774622587597925
replaying at 609 xgboost
FPS:  31.712566157568425
replaying at 610 xgboost
FPS:  442.67060686015833
replaying at 611 xgboost
FPS:  292.0621126662489
replaying at 612 xgboost
FPS:  35.18032594381957
replaying at 613 xgboost
FPS:  72.82409931417658
replaying at 614 xgboost
FPS:  26.575326148250934
replaying at 615 xgboost
FPS:  385.4704530833563
replay

...

ZMQError: Too many open files

ZMQError: Too many open files

567.5648173207037
replaying at 859 xgboost
FPS:  Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-0189f95c09f1>", line 1, in <module>
    steer(True)
  File "<ipython-input-10-bf33f4a4271d>", line 153, in steer
    print("FPS: ", 1.0 / (time.time() - start))
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 352, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 186, in schedule
    event_id = os.urandom(16)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-0189f95c09f1>", line 1, in <module>
    steer(True)
  File "<ipython-input-10-bf33f4a4271d>", line 153, in steer
    print("FPS: ", 1.0 / (time.time() - start))
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 352, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 186, in schedule
    event_id = os.urandom(16)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/IPython/core/in

ZMQError: Too many open files

In [59]:
data = np.random.rand(100,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=100)
dtest = xgb.DMatrix([data[0]])
print(label.shape)
DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)
# np.argmax([DQNAgent().bst.predict(dtest),10])

(100,)
[0]	train-merror:0.41	eval-merror:0.41
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.33	eval-merror:0.33
[2]	train-merror:0.25	eval-merror:0.25
[3]	train-merror:0.25	eval-merror:0.25
[4]	train-merror:0.26	eval-merror:0.26
[5]	train-merror:0.25	eval-merror:0.25
[6]	train-merror:0.19	eval-merror:0.19
[7]	train-merror:0.19	eval-merror:0.19
[8]	train-merror:0.18	eval-merror:0.18
[9]	train-merror:0.16	eval-merror:0.16
[10]	train-merror:0.17	eval-merror:0.17
[11]	train-merror:0.15	eval-merror:0.15
[12]	train-merror:0.15	eval-merror:0.15
[13]	train-merror:0.13	eval-merror:0.13
[14]	train-merror:0.13	eval-merror:0.13
[15]	train-merror:0.13	eval-merror:0.13
[16]	train-merror:0.13	eval-merror:0.13
[17]	train-merror:0.12	eval-merror:0.12
[18]	train-merror:0.13	eval-merror:0.13
[19]	train-merror:0.12	eval-merror:0.12
[20]	train-merror:0.13	eval-merror:0.13
[21]	train-merro

array([ 0.5], dtype=float32)